# MultiModal Kinetic Modeling

In this tutorial we will investigate how to simultaneously optimize transient optical and transient X-ray emission spectroscopy 

In [71]:
import os,sys
import pandas as pd
import numpy as np
import matplotlib,lmfit
import matplotlib.pyplot as plt
import KiMoPack.plot_func as pf
from importlib import reload
reload(pf)
%matplotlib tk

Plot_func version 7.6.14
was imported from path:
 C:\Users\jensu\anaconda3\Lib\site-packages\KiMoPack
The current working folder is:
 c:\Users\jensu\Dropbox\coding\github\KiMoPack\Tutorial_Notebooks


## 1) Load Data


In [70]:
filepath = os.path.join(os.getcwd(), 'Data', 'MultiModal')  # set path to file to fit

Prepare the transient optical data

In [72]:
ta=pf.TA(filename='combined_optical_spectrum.SIA',path=filepath)  #load transient optical
plt.close('all')
ta.Background()                                              # Remove the background before t=0
ta.wavelength_bin=20                                        # width of wavelength bin in kinetics
ta.wave_nm_bin=5;
ta.ignore_time_region
ta.rel_wave=np.concatenate([np.arange(300,572+25,25),np.arange(662,800,25)])# rebinning wavelength
ta.timelimits=[-0.2,100]                                     # use to manually limit the range to plot (and fit)     
ta.bordercut=[335,766]                                      # use to set the outer wavelength limits
ta.scattercut=[572,662]                                     # this region is blanked out to block scatter, this can be a list of regions
ta.intensity_range=1.2e-2                          # set a value 5e-3 or a range [-1e-3,5e-3] for 2d plotting
ta.Plot_RAW()                                      # Plot Raw Spectra

In [73]:
import function_library as func

In [137]:
plt.close('all')                                             # Close old spectra
ta.mod=func.P12                                        # Choose the model 'exponential', 'consecutive' and 'fast_consecutive'are build in
par=lmfit.Parameters()                                       # create empty parameter object

par.add('k0',value=1/0.275,vary=True)                                # Add second rate component
par.add('k1',value=1/7.2,vary=True)                                # Add one rate component
par.add('k2',value=1/3,vary=True)                                 # Add second rate component
par.add('t0',value=-0.0167,min=-2,max=2,vary=False)                       # Allow the arrival time to adjust (for all)
par.add('resolution',value=0.0935,min=0.04,max=0.5,vary=False)       # Allow the instrument response to adjust (for all)
#par.add('explicit_GS')
par.add('infinite')                                                # Keyword for an non decaying component
#par.add('background')                                              # Keyword for an non decaying component                                             # Keyword to fit the Background Disable the background substraction in the loading cell for best effect
ta.ignore_time_region=[-0.5,0.3]                                      # select time window in which to fit
ta.log_fit=False                                               # fit in linear or log scale
ta.timelimits=[-1,200]
if 0:
    for key in par.keys():
        par[key].vary=False
ta.par=par                                                     # write parameter object into file for fitting
ta.Fit_Global()                                 # trigger fitting

ta.error_matrix_amplification=10;                              # Choose how much stronger should error be plotted       
ta.log_scale=False
ta.Plot_fit_output([1,4])# 
#ta.Plot_fit_output(range(5))                                 # plot the fit output

0.0009240314078486792
0.0009630361379469789
0.0009181588077512039
0.000929327855587112
0.0008817128570778466
0.0008451153446832068
0.0008687314710753901
0.0008609014874985881
0.0007971343828661734
0.0007438227482161941
0.0007624585661392267
0.0006954646872159992
0.0006186460593638737
0.0006206725482638144
0.0005729015894529144
0.000586154386487148
0.0005868243103055885
0.0005282952753963553
0.0005238693491806496
0.0007054303108913236
0.0005574567488819824
0.0005948861251897351
0.0005469459072633384
0.0005256843462699444
0.0005820755392123382
0.0005306204164573509
0.000553167270414904
0.0005287223802757367
0.0005420414079703203
0.00052357454811601
0.0005340554290680495
0.0005236071925026403
0.000527031677720228
0.0005229823959446277
0.0005268571455840549
0.0005224520389936705
0.0005225196537073779
0.0005287864765961352
0.0005221185185314683
0.0005221015785370966
0.0005230231989914632
0.000522802284845069
0.0005220892538490752
0.0005226417322725738
0.0005220715495122436
0.000522317360704

In [75]:
plt.close('all')

These results are terrible, mainly because the optical artifacts in the data are disturbing the system. Lets see what the XES is doing.

In [76]:
xes=pf.TA('XES_on.SIA',path=filepath,sep=',',units='eV',sort_indexes=True,decimal='.',data_type='X-ray emission intensity',baseunit='ps')   #
plt.close('all')
xes.ds=pf.Frame_golay(xes.ds,7,1)
xes.rel_wave=np.arange(7053,7061,1)      # use to manually select interesting wavelength for the kinetics
xes.wavelength_bin=1                                        # width of wavelength bin in kinetics
xes.rel_time=[-0.5,-0.1,0,0.1,0.3,0.5,1,1.5,3,5,10,15,20]      # use to manually set interesting times for the spectra
xes.time_width_percent=0                                    # rebinning of time_points in percent
xes.timelimits=[-1,200]                                     # use to manually limit the range to plot (and fit)     
xes.log_scale=False                                         # use to plot the 2d plots with logarithmic intensity scale
xes.bordercut=[7045,7065]                                      # use to set the outer wavelength limits
xes.intensity_range=[0,7]                           # set a value 5e-3 or a range [-1e-3,5e-3] for 2d plotting
xes.Plot_RAW()                                      # Plot Raw Spectra

In [77]:
plt.close('all')                                            
xes.mod=func.P12                                       
par=ta.par_fit                                       
par['t0'].value=-0.0167                               
par['resolution'].value=0.0935
#par.add('explicit_GS')
par.add('infinite')                                                # Keyword for an non decaying component
par.add('background')                                             # Keyword to fit the Background Disable the background substraction in the loading cell for best effect

if 0:
    for key in par.keys():
        par[key].vary=False
xes.par=par                                                     # write parameter object into file for fitting
xes.Fit_Global()                                 # trigger fitting
xes.error_matrix_amplification=1;                              # Choose how much stronger should error be plotted                                                    # 2D plots in linear or log scale
xes.Plot_fit_output()                                 # plot the fit output

63.19231553376781
63.01379912457283
62.34136080889739
63.09374311975552
62.41344032130324
62.10623863376638
61.69753396885842
61.492854706007286
61.0482967357607
61.12044989552248
60.59572444425639
60.29697362635967
60.53413871515726
60.35880255304349
60.72325302439644
60.46326603238304
60.19627959681306
60.104324052337546
60.017679483318666
59.936758452506766
59.73036908436961
59.513696009388774
59.26157797577565
58.88008425440181
59.66268673304957
58.54034933500386
58.13848904676965
58.687399093212726
58.123627537447504
58.67795874317338
57.737882054309054
57.74983914856882
57.66615223173419
58.57650671696372
58.330633214943674
57.76881853394413
58.106809645059606
57.707805575986676
57.60426006076189
57.761990719667736
58.596812129825274
57.58323712952035
57.59099997627049
57.70304738105841
57.58852945412626
57.60352315409182
57.57588676672823
57.644985481790634
57.56957987594349
57.64096616292831
57.569039748374806
57.57125326840819
57.58609504668808
57.567974363273194
57.5745741927

In [78]:
xes.Background(uplimit=-0.25)
xes.intensity_range=1
try:
    xes.par.pop('background')
except:
    pass
xes.Fit_Global()
plt.close('all')
xes.Plot_fit_output()

64.88585719479536
64.70897955127022
64.03789575157903
64.78818219575264
64.11072570611955
63.80451744832317
63.397773081894954
63.193599233674526
62.75168638869167
62.82312846275994
62.3035478702676
62.01059984640254
62.246316435925564
62.075689709601264
62.44872932155223
62.18303951480333
61.9179122378805
61.83107298381194
61.73552473101766
61.65324974663188
61.45106537048635
61.23591461486093
60.992199457053815
60.61963114635462
61.38471019147066
60.279767209738395
59.889195287048565
60.44311767330389
59.90226731340117
59.522429673123455
59.555798454023176
59.4465899813237
60.361474530562646
60.27315014837066
59.552246869274924
59.79600912757093
59.49614560553835
59.391650840215675
59.56800053180679
59.56857812060785
59.4003359815284
59.56733782113751
59.38720813414726
59.6781109718101
59.36168216118038
59.42215159141108
59.369592421238735
59.40098901451436
59.36740692356386
59.374351435399426
59.35962253554949
59.401173834978486
59.35788305341255
59.36616745271721
59.35873603402952


In [161]:
weights=[1/ta.re['A'].abs().sum().sum(),1/xes.re['A'].abs().sum().sum()]
print(weights)
ta1=ta.Copy()
xes1=xes.Copy()

[0.05734682804498479, 0.00022052678112150292]


In [160]:
ta1.par=ta.par_fit
ta1.Fit_Global(multi_project=[xes1],same_DAS=False,same_shape_params=False,weights=weights,use_ampgo=True)
ta1.Plot_fit_output()

5615725.376481927
5473675.486102197
5456777.919891555
5536794.947548318
5364480.1560748
5246504.5287390305
5248992.491992482
5173800.642282268
5046879.611945996
4923024.924081714
4700573.52363646
4747716.434358571
4506926.167490962
4277935.463678575
4192593.1312632263
3977751.900916092
3981699.272936708
3920955.617455199
4081506.0702803005
3893915.3302946794
4056209.191789563
3920393.9873901024
4146374.874247906
3888754.6500073317
3864608.6400909596
3855287.6063541896
3825728.6823545913
3826876.9914918034
3819864.1248689797
3858205.746326648
3801700.848564923
3831372.043913698
3840164.3833014728
3808252.517018879
3788037.6456613303
3805829.63611646
3795122.788863535
3836609.838140168
3788880.124979644
3804364.128427069
3790150.750198995
3803207.0566103905
3787821.965730681
3793180.772942357
3786842.714604125
3790680.91048046
3786530.5792872277
3792565.228983528
3786178.465353847
3788975.889431887
3786424.845128294
3786787.1982216155
3786212.681917671
3786647.9186971914
3786134.86987306

In [122]:
xes1.par=ta1.par_fit
for key in xes1.par.keys():
    xes1.par[key].vary=False
xes1.Fit_Global()
plt.close('all')
xes1.Plot_fit_output()

ATTENTION: we have not optimized anything but just returned the parameters

Fit Results:
Model Used: External function

The minimum error is:6.02160524e+01
The minimum R2-value is:9.81073932e-01

In Rates

+------------+-----------+--------------+--------+---------+-------+--------+
|            |     value |   init_value | vary   |     min |   max | expr   |
+============+===========+==============+========+=========+=======+========+
| k0         |  2.17907  |     2.17907  | False  |    0    | inf   |        |
+------------+-----------+--------------+--------+---------+-------+--------+
| k1         |  0.259199 |     0.259199 | False  |    0    | inf   |        |
+------------+-----------+--------------+--------+---------+-------+--------+
| k2         |  0.921685 |     0.921685 | False  |    0    | inf   |        |
+------------+-----------+--------------+--------+---------+-------+--------+
| t0         | -0.0167   |    -0.0167   | False  |   -2    |   2   |        |
+------------+